### Import statements, Jupyter magic y definimos constantes

In [1]:
%matplotlib inline

In [2]:
from enum import Enum
from math import floor, sqrt
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np

class PaddingMode(Enum):
    CEROS = "constant"
    REPLICA = "edge"
    SIMETRIA = "symmetric"
    CICLICO = "wrap"

### Funciones auxiliares

In [5]:
def apply_kernel(image, kernel, post_kernel_operation):
    
    border_offset = floor(len(kernel) / 2)
    old_y_size, old_x_size = image.shape
    
    # The new image doesn't have a border and we need to account for the first and last row and columns (x2)
    new_image = np.zeros((old_y_size - (border_offset * 2), old_x_size - (border_offset * 2)), dtype=int)
    
    # Apply the kernel to each of the values
    # We need to start at the center of the kernel on the image, which is given by (boder_offset, border_offset)
    #  And end at the maximum size minus the border, with that, the output image will not have a border
    for y in range(border_offset, old_y_size - border_offset):
        for x in range(border_offset, old_x_size - border_offset):
            
            new_pixel = 0
            for (kernel_x, kernel_y), kernel_value in np.ndenumerate(kernel):
                old_value = image[x - border_offset + kernel_x,
                                  y - border_offset + kernel_y]
                new_pixel += old_value * kernel_value
            
            new_pixel = post_kernel_operation(new_pixel)
            
            new_x = x - border_offset
            new_y = y - border_offset
            
            new_image[new_x, new_y] = new_pixel
            
    return new_image

def scale_pixel_values(image, low: int = 0, high: int = 255):
    image = np.copy(image).astype("int32")
    
    min_value = np.amin(image)
    max_value = np.amax(image)
    
    image_scale_size = max_value - min_value
    reference_scale_size = high - low
    
    scale_factor = image_scale_size / reference_scale_size
    scale_factor = 1 if scale_factor == 0 else scale_factor # Avoiding dividing by zero
    
    # Use the size factor to divide all of the values in the image and floor them
    y_size, x_size = image.shape
    
    for y in range(y_size):
        for x in range(x_size):
            # First, we need to move the value to begin at the same point as low
            val = image[y][x]
            moved_val = val - min_value + low
            # Apply the floored scale factor
            image[y][x] = floor(moved_val / scale_factor)
    
    image = image.astype("int32")
    
    return image

def execute_operator(image, kernel,
                     is_convolution: bool=False,
                     padding_mode: PaddingMode = PaddingMode.CEROS,
                     post_kernel_operation = lambda pixel: pixel):
    # TODO: Add option to apply Gaussian blur prior to applying the kernel
    # We are assuming the kernel will be a square with a single center, i.e. size is of odd length
    image = np.copy(image).astype("int32")
    if is_convolution:
        kernel = np.rot90(kernel, k=2)
    
    # Calculate the padding
    border_offset = floor(len(kernel) / 2)
    
    # Add padding to make sure the kernel is uniformly applied
    if padding_mode == PaddingMode.CEROS:
        image = np.pad(image, pad_width=border_offset, mode=PaddingMode.CEROS.value, constant_values=0)
    else:
        image = np.pad(image, pad_width=border_offset, mode=padding_mode.value)
    
    image = image.astype("int32")
    
    new_image = apply_kernel(image, kernel, post_kernel_operation)
    
    return new_image

def combine_gradients(gx, gy, strategy):
    y_len, x_len = gx.shape
    gradient = np.zeros((y_len, x_len))
    
    # TODO: Calculate Gradient Direction Too
    for y in range(y_len):
        for x in range(x_len):
            gradient[y][x] = strategy(gx[y][x], gy[y][x])
    
    return gradient

def calculate_gradient_pythagoras(x_elm, y_elm):
    return sqrt(x_elm**2 + y_elm**2)

def convert_to_bw_gradient_image(image, threshold: int):
    new_image = np.copy(image).astype("int32")
    new_image[new_image >= threshold] = 255
    new_image[new_image != 255] = 0
    return new_image

### 1. Cargar la imagen

In [8]:
w = [[-10,-10, 0],
     [-10,  0, 10],
     [ 0,  10, 10]]

H = [[10,20,0,30],
     [20,0,10,20],
     [10,0,40,10],
     [20,40,0,30]]

In [9]:
# Cross Correlation
ccorr = execute_operator(H, w, is_convolution = False)
print(ccorr)

[[ 400    0  400  200]
 [   0    0  500 -300]
 [ 400  500  300 -400]
 [ 300 -300 -500 -500]]


In [10]:
# Convolution
ccorr = execute_operator(H, w, is_convolution = True)
print(ccorr)

[[-400    0 -400 -200]
 [   0    0 -500  300]
 [-400 -500 -300  400]
 [-300  300  500  500]]


In [6]:
# Ejercicio Cross Correlation Clase
w_cc1 = [[-1, 0, 1],
         [-1, 0, 1],
         [-1, 0, 1]]

f_cc1 = [[2, 1, 3],
         [5, 0, 4],
         [1, 6, 5]]

f_cc_w = execute_operator(f_cc1, w_cc1, is_convolution = False)

print(f_cc_w)
print()

w_cc_f = execute_operator(w_cc1, f_cc1, is_convolution = False)

print(w_cc_f)


[[ 1  0 -1]
 [ 7  4 -7]
 [ 6  3 -6]]

[[-6  3  6]
 [-7  4  7]
 [-1  0  1]]


In [7]:
# Ejercicio convolution Clase
w_con1 = [[-1, 0, 1],
         [-1, 0, 1],
         [-1, 0, 1]]

f_con1 = [[2, 1, 3],
         [5, 0, 4],
         [1, 6, 5]]

f_con_w = execute_operator(f_con1, w_con1, is_convolution = True)

print(f_con_w)
print()

w_con_f = execute_operator(w_con1, f_con1, is_convolution = True)

print(w_con_f)



[[-1  0  1]
 [-7 -4  7]
 [-6 -3  6]]

[[-1  0  1]
 [-7 -4  7]
 [-6 -3  6]]
